In [12]:
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder

In [13]:
df = pd.read_csv('/content/haunted_places.csv')
df.head(2)

,city,country,description,location,state,state_abbrev,longitude,latitude,city_longitude,city_latitude
0,Ada,United States,Ada witch - Sometimes you can see a misty blue...,Ada Cemetery,Michigan,MI,-85.504893,42.962106,-85.495480,42.960727
1,Addison,United States,A little girl was killed suddenly while waitin...,North Adams Rd.,Michigan,MI,-84.381843,41.971425,-84.347168,41.986434


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10992 entries, 0 to 10991
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   city            10989 non-null  object 
 1   country         10992 non-null  object 
 2   description     10992 non-null  object 
 3   location        10989 non-null  object 
 4   state           10992 non-null  object 
 5   state_abbrev    10992 non-null  object 
 6   longitude       9731 non-null   float64
 7   latitude        9731 non-null   float64
 8   city_longitude  10963 non-null  float64
 9   city_latitude   10963 non-null  float64
dtypes: float64(4), object(6)
memory usage: 858.9+ KB


In [15]:
df.shape

(10992, 10)

In [16]:
df.city.value_counts()

Los Angeles           61
San Antonio           55
Honolulu              43
Pittsburgh            42
Columbus              41
                      ..
Swedesboro             1
Tabernacle             1
Toms River             1
Tewksbury Township     1
Woodland Park          1
Name: city, Length: 4385, dtype: int64

In [17]:
df.state.value_counts()

California        1070
Texas              696
Pennsylvania       649
Michigan           529
Ohio               477
New York           459
Illinois           395
Kentucky           370
Indiana            351
Massachusetts      342
Florida            328
Missouri           314
Georgia            289
Wisconsin          274
Alabama            224
Tennessee          221
Washington         218
North Carolina     211
Oklahoma           211
New Jersey         194
Virginia           192
Connecticut        185
Maryland           170
Iowa               169
South Carolina     168
Colorado           166
Minnesota          162
Arizona            156
Oregon             155
Utah               134
Kansas             123
Arkansas           119
Nebraska           111
West Virginia      106
Louisiana           99
Hawaii              96
New Mexico          78
Wyoming             76
South Dakota        75
New Hampshire       72
Rhode Island        68
Idaho               66
Vermont             63
Mississippi

In [22]:
# Asumiendo que 'df' es tu DataFrame original
haunted_places = df

# Crear un LabelEncoder separado para cada columna
label_encoder_city = LabelEncoder()
label_encoder_state = LabelEncoder()
label_encoder_country = LabelEncoder()
label_encoder_state_abbrev = LabelEncoder()

# Aplicar la codificación de etiquetas a las columnas categóricas 'city', 'state', 'country', y 'state_abbrev'
haunted_places['city'] = label_encoder_city.fit_transform(haunted_places['city'])
haunted_places['state'] = label_encoder_state.fit_transform(haunted_places['state'])
haunted_places['country'] = label_encoder_country.fit_transform(haunted_places['country'])
haunted_places['state_abbrev'] = label_encoder_state_abbrev.fit_transform(haunted_places['state_abbrev'])

# Seleccionar solo las columnas numéricas (excluyendo 'location')
columns_to_impute = ['state_abbrev', 'longitude', 'latitude', 'city_longitude', 'city_latitude']

# Crear un imputador k-NN
imputer = KNNImputer(n_neighbors=5)

# Imputar los valores faltantes en las columnas numéricas
haunted_places[columns_to_impute] = imputer.fit_transform(haunted_places[columns_to_impute])

# Revertir la codificación de etiquetas para las columnas categóricas
haunted_places['city'] = label_encoder_city.inverse_transform(haunted_places['city'].round(0).astype(int))
haunted_places['state'] = label_encoder_state.inverse_transform(haunted_places['state'].round(0).astype(int))
haunted_places['country'] = label_encoder_country.inverse_transform(haunted_places['country'].round(0).astype(int))
haunted_places['state_abbrev'] = label_encoder_state_abbrev.inverse_transform(haunted_places['state_abbrev'].round(0).astype(int))
haunted_places.head()

,city,country,description,location,state,state_abbrev,longitude,latitude,city_longitude,city_latitude
0,Ada,United States,Ada witch - Sometimes you can see a misty blue...,Ada Cemetery,Michigan,MI,-85.504893,42.962106,-85.495480,42.960727
1,Addison,United States,A little girl was killed suddenly while waitin...,North Adams Rd.,Michigan,MI,-84.381843,41.971425,-84.347168,41.986434
2,Adrian,United States,If you take Gorman Rd. west towards Sand Creek...,Ghost Trestle,Michigan,MI,-84.035656,41.904538,-84.037166,41.897547
3,Adrian,United States,"In the 1970's, one room, room 211, in the old ...",Siena Heights University,Michigan,MI,-84.017565,41.905712,-84.037166,41.897547
4,Albion,United States,Kappa Delta Sorority - The Kappa Delta Sororit...,Albion College,Michigan,MI,-84.745177,42.244006,-84.753030,42.243097


In [29]:
from collections import Counter

# Concatenar todas las descripciones en un solo texto
all_descriptions = ' '.join(haunted_places['description'])

# Dividir el texto en palabras individuales
words = all_descriptions.split()

# Contar la frecuencia de cada palabra
word_count = Counter(words)

# Encontrar las 10 palabras más comunes
most_common_words = word_count.most_common(10)

# Imprimir las 10 palabras más comunes
for word, count in most_common_words:
    print(f"Palabra: '{word}' - Ocurrencias: {count}")


Palabra: 'the' - Ocurrencias: 53571
Palabra: 'and' - Ocurrencias: 23252
Palabra: 'a' - Ocurrencias: 21846
Palabra: 'of' - Ocurrencias: 19532
Palabra: 'in' - Ocurrencias: 16707
Palabra: 'to' - Ocurrencias: 14445
Palabra: 'is' - Ocurrencias: 12977
Palabra: 'that' - Ocurrencias: 9211
Palabra: 'was' - Ocurrencias: 8888
Palabra: 'on' - Ocurrencias: 8383


In [32]:
import nltk
from nltk import sent_tokenize
from collections import Counter

# Descargar los datos necesarios para NLTK si aún no lo has hecho
nltk.download('punkt')

# Concatenar todas las descripciones en un solo texto
all_descriptions = ' '.join(haunted_places['description'])

# Dividir el texto en oraciones
sentences = sent_tokenize(all_descriptions)

# Contar la frecuencia de cada oración
sentence_count = Counter(sentences)

# Encontrar las 10 oraciones más comunes
most_common_sentences = sentence_count.most_common(10)

# Imprimir las 10 oraciones más comunes
for sentence, count in most_common_sentences:
    print(f"Oración: '{sentence}' - Ocurrencias: {count}")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Oración: '.' - Ocurrencias: 11
Oración: 'Apparitions seen.' - Ocurrencias: 5
Oración: 'Feelings of being watched.' - Ocurrencias: 5
Oración: 'No Trespassing.' - Ocurrencias: 4
Oración: 'Cold spots felt.' - Ocurrencias: 4
Oración: '!' - Ocurrencias: 4
Oración: '-  A WARNING TO ALL THAT INVESTIGATE THE AREA.' - Ocurrencias: 3
Oración: 'TRESSPASSERS ARE NOT ALLOWED.' - Ocurrencias: 3
Oración: 'THERE IS A SECURITY SERVICE THAT WILL ESCORT YOU OFF THE PREMISES IF FOUND.' - Ocurrencias: 3
Oración: 'Be careful.' - Ocurrencias: 3


In [21]:
haunted_places.location.value_counts()

Prince Georges county            18
Cemetery                         13
Cry Baby Bridge                  13
Oviedo                           12
Wal-Mart                         12
                                 ..
Dale Mansion Site                 1
Lilly Lane                        1
The Carter Mansion                1
Hell's Gates (Green Mountain)     1
Saylor Park Forest                1
Name: location, Length: 9903, dtype: int64